### Azimuthal Wind Timeseries

Emily Luschen \
Emily.W.Luschen-1@ou.edu \
created: 29 Jan 2025

Gameplan: (edit: 1 May 2025)
1. Load in u, v, lat, lon
2. Mask u, v
3. Calculate az wind
3.1 Bin az wind by radius
4. Find max wind
5. Repeat for each test (do ctl seperately due to change in time)
7. Pickle the winds

In [1]:
# import packages
import numpy as np # libary for working with arrays
from grl_mask_tc_track_3d import mask_tc_track_3d # function to mask out the TC track
import grl_functions as fun # custom functions
from grl_azmuth_wind_level import azim_wind_latlon
import xarray as xr
import pickle

Sensitivity Tests

In [2]:
storm = 'haiyan' # or 'maria'

if storm == 'haiyan':
    wrfout = wrfpath # for lat and lon
    ntest = 4
    tests = ['ncrf36h', 'STRATANVIL_OFF', 'STRATANVIL_ON', 'STRAT_OFF']
    nmem = 8
elif storm == 'maria': # maria
    wrfout = wrfpath_maria # for lat and lon
    ntest = 1
    tests = ['ncrf48h'] 
    nmem = 10


# data parameters
p_lev = 0 # only one level available
t0 = 0 # start time index
t1 = 49 # end time index
ntime = 49 # senstivity test time length
membs = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

# TC tracking parameters
ptrack='600' # tracking pressure level
var_track = 'rvor' # variable
rmax = 3 # radius (deg) limit for masking around TC center

In [3]:
# get the lat and lon

varname='XLAT'
lat = fun.var_wrfread(wrfout,varname) # latitude
varname='XLONG'
lon = fun.var_wrfread(wrfout,varname) # latitude

nlat, nlon = np.shape(lat)

### Sensitivity Test Loop

In [4]:
# initialize arrays
azwind_plt = np.zeros((ntest, ntime))*np.nan # test x time (4,49)
azwind_membs = np.zeros((ntest,nmem,ntime))*np.nan # test x mem x time

max_radius = rmax*111# [km]; find max radius
bin_radius = np.arange(0,max_radius+10,10) # radius bins with 10 km interval


for s in range(0,ntest): # 6. Do for each test

    print('Test: ', tests[s])

    track_t0 = 36
    track_t1 = track_t0 + ntime

    for m in range(0,nmem):
        # 1. load data
        ufile = U_path # sfc wind
        vfile = V_path # sfc wind
        df = xr.open_mfdataset([ufile, vfile])
        u_wind = df.U10.data[:,p_lev,:,:]
        v_wind = df.V10.data[:,p_lev,:,:]

        # 2. mask winds
        track_file = tracking_file_path # tracking file
        uwind_mask = mask_tc_track_3d(track_file, rmax, u_wind, lon, lat, t0=t0, t1=t1)
        vwind_mask = mask_tc_track_3d(track_file, rmax, v_wind, lon, lat, t0=t0, t1=t1)

        # 3. calculate azimuthal wind and radius
        r_wind, az_wind, theta, radius = azim_wind_latlon(track_file, uwind_mask, vwind_mask, lon=lon, lat=lat, t0=t0, t1=t1)

        # binning for radius
        radius_km = radius*111 # [km]; 111 km per degree
        r_wind_azavg = np.zeros((ntime,len(bin_radius)-1))*np.nan
        az_wind_azavg = np.zeros((ntime,len(bin_radius)-1))*np.nan
        for t in range(ntime):
            for b in range(0,len(bin_radius)-1):
                bidx_radius = (radius_km[t,:,:]>=bin_radius[b])&(radius_km[t,:,:]<bin_radius[b+1]) # index for radius bin
                r_wind_azavg[t,b] = np.ma.mean(r_wind[t,bidx_radius]) # mean radial wind in each radius bin
                az_wind_azavg[t,b] = np.ma.mean(az_wind[t,bidx_radius]) # mean azimuthal wind in each radius bin
        
        # 4. Find max azimuthal wind
        azwind_membs[s,m,:] = np.ma.max(az_wind_azavg, axis=1) # membs x time

        print('Member: ', membs[m])

    print('Test: ', tests[s])
# estimated completion time: 26m 19s

Test:  ncrf36h
Member:  01
Member:  02
Member:  03
Member:  04
Member:  05
Member:  06
Member:  07
Member:  08
Test:  ncrf36h
Test:  STRATANVIL_OFF
Member:  01
Member:  02
Member:  03
Member:  04
Member:  05
Member:  06
Member:  07
Member:  08
Test:  STRATANVIL_OFF
Test:  STRATANVIL_ON
Member:  01
Member:  02
Member:  03
Member:  04
Member:  05
Member:  06
Member:  07
Member:  08
Test:  STRATANVIL_ON
Test:  STRAT_OFF
Member:  01
Member:  02
Member:  03
Member:  04
Member:  05
Member:  06
Member:  07
Member:  08
Test:  STRAT_OFF


### CTL Loop

In [5]:
ntime = 97 
track_t0 = 0
track_t1 = 97

azwind_membs_ctl = np.zeros((nmem, ntime))*np.nan # memb x time (8,49)

for m in range(0,nmem):
    # 1. load data
    ufile = U_path # sfc wind
    vfile = V_path # sfc wind
    df = xr.open_mfdataset([ufile, vfile])
    u_wind = df.U10.data[:,p_lev,:,:] 
    v_wind = df.V10.data[:,p_lev,:,:]

    # 2. mask winds
    track_file = tracking_file_path # tracking file
    uwind_mask = mask_tc_track_3d(track_file, rmax, u_wind, lon, lat, t0=track_t0, t1=track_t1)
    vwind_mask = mask_tc_track_3d(track_file, rmax, v_wind, lon, lat, t0=track_t0, t1=track_t1)

    # 3. calculate azimuthal wind and radius
    r_wind, az_wind, theta, radius = azim_wind_latlon(track_file, uwind_mask, vwind_mask, lon=lon, lat=lat, t0=track_t0, t1=track_t1)

    # binning for radius
    radius_km = radius*111 # [km]; 111 km per degree
    r_wind_azavg = np.zeros((ntime,len(bin_radius)-1))*np.nan
    az_wind_azavg = np.zeros((ntime,len(bin_radius)-1))*np.nan
    for t in range(ntime):
        for b in range(0,len(bin_radius)-1):
            bidx_radius = (radius_km[t,:,:]>=bin_radius[b])&(radius_km[t,:,:]<bin_radius[b+1]) # index for radius bin
            r_wind_azavg[t,b] = np.ma.mean(r_wind[t,bidx_radius]) # mean radial wind in each radius bin
            az_wind_azavg[t,b] = np.ma.mean(az_wind[t,bidx_radius]) # mean azimuthal wind in each radius bin
    
    # 4. Find max azimuthal wind
    azwind_membs_ctl[m,:] = np.ma.max(az_wind_azavg, axis=1)

    print('Member: ', membs[m])

# estimated completion time: 13m 46s

/scratch/eluschen/tmp/ipykernel_45913/3454413697.py:36: UserWarning: Warning: converting a masked element to nan.
  r_wind_azavg[t,b] = np.ma.mean(r_wind[t,bidx_radius]) # mean radial wind in each radius bin
/scratch/eluschen/tmp/ipykernel_45913/3454413697.py:37: UserWarning: Warning: converting a masked element to nan.
  az_wind_azavg[t,b] = np.ma.mean(az_wind[t,bidx_radius]) # mean azimuthal wind in each radius bin


Member:  01
Member:  02
Member:  03
Member:  04
Member:  05
Member:  06
Member:  07
Member:  08


In [10]:
# Write pickle file
pickle_file = output_file_path
with open(pickle_file, 'wb') as file:
    pickle.dump([azwind_membs, azwind_membs_ctl], file)
    # azwind_membs: test x memb x time (4 x 8 x 49)
    # azwind_membs_ctl: memb x time (8 x 97)